# Building a GenAI RAG application with Feature Store and BigQuery

## Overview
This notebook guides you through building a low-latency vector search system for your GenAI application using Vertex AI Feature Store. We'll leverage the [Vertex Feature Store Langchain integration]([link to integration]) to streamline this process.

Feature Store seamlessly integrates with BigQuery, providing a unified data storage and flexible vector search options:

- **BigQuery Vector Search**: Ideal for batch retrieval and prototyping, as it requires no infrastructure setup.
- **Feature Store Online Store**: Enables low-latency retrieval with manual or scheduled data sync. Perfect for production-ready user-facing GenAI applications.

![Image notebook journey](diagram_journey.png)


# Setup


### Install libraries

In [ ]:
!pip install langchain-google-vertexai pypdf==4.2.0 langchain pyarrow==16.0.0 db-dtypes==1.2.0 --upgrade

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

In [1]:
%load_ext autoreload
%autoreload 2
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_vertexai.vectorstores.feature_store.featurestore import VertexFSVectorStore
from langchain_google_vertexai.vectorstores.feature_store.bigquery import BigQueryVectorStore

### Define environment variables

In [2]:
PROJECT_ID = "cloud-llm-preview2"
DATASET = "vertex_documentation"
TABLE = "mytest99"
REGION = "europe-west4"

# Add documents to `BigQueryVectorStore`

This step ingests and parse PDF documents, split them, generate embeddings and add the embeddings to the vector store. The document corpus used as dataset is a collection of owners car manual.

**Summary steps**
- Create text embeddings: LangChain `VertexAIEmbeddings`
- Ingest PDF files: LangChain `PyPDFLoader`
- Chunk documents: LangChain `TextSplitter`
- Create Vector Store: LangChain  `VertexAIFeatureStore` 

### Create the VertexAI Embedding model

In [3]:
embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### Ingest PDF file

The document is hosted on Cloud Storage bucket (at `gs://github-repo/generative-ai/sample-apps/fixmycar/cymbal-starlight-2024.pdf`) and LangChain provides a convenient document loader [`PyPDFLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/) to load documents from pdfs.


In [ ]:
GCS_BUCKET_DOCS = (
    "github-repo/generative-ai/sample-apps/fixmycar"  # @param {type: "string"}
)

# Copy the file to the current path
!gsutil cp "gs://$GCS_BUCKET_DOCS/*.pdf" .

In [ ]:
# Ingest PDF files
loader = PyPDFLoader("cymbal-starlight-2024.pdf")
documents = loader.load()

# Add document name and source to the metadata
for document in documents:
    doc_md = document.metadata
    document_name = doc_md["source"].split("/")[-1]
    # derive doc source from Document loader
    doc_source_prefix = "/".join(GCS_BUCKET_DOCS.split("/")[:3])
    doc_source_suffix = "/".join(doc_md["source"].split("/")[4:-1])
    source = f"{doc_source_prefix}/{doc_source_suffix}"
    document.metadata = {"source": source, "document_name": document_name}

print(f"# of documents loaded (pre-chunking) = {len(documents)}")

Verify document metadata

In [ ]:
documents[0].metadata

## Chunk documents - TextSplitter

Split the documents to smaller chunks. When splitting the document, ensure a few chunks can fit within the context length of LLM.

In [ ]:
# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
for idx, split in enumerate(doc_splits):
    split.metadata["chunk"] = idx

print(f"# of documents = {len(doc_splits)}")

In [ ]:
doc_splits[0].metadata

## Configure `BigQueryVectorStore` as Vector Store

You are now ready to use BigQuery Vector Store!

You can initialize the class by providing:
- `project_id`
- `location`
- `dataset_name`
- `table_name`

The table will be used to store embeddings and metadata. You can also point to an existing table. 

The class will use [BigQuery Vector Search](https://cloud.google.com/bigquery/docs/vector-search-intro) to perform vector search.

See [here](TODO) for the full list of parameters of the class. 

In [4]:
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    location=REGION,
    dataset_name=DATASET,
    table_name=TABLE,
    embedding=embedding_model,
 )

BigQuery table cloud-llm-preview2.vertex_documentation.mytest99 initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=cloud-llm-preview2&ws=!1m5!1m4!4m3!1scloud-llm-preview2!2svertex_documentation!3smytest99
BigQueryVectorStore initialized with BigQuery VectorSearch. 
Optional online serving available via .to_vertex_fs_vector_store() method.


### Add documents to the store

Note: If you have precomputed embeddings, you can add text, embeddings and potential metadata using the method `add_texts_with_embeddings`

In [ ]:
bq_store.add_documents(doc_splits)

Verify the BigQueryVectorSearch with similarity search

In [5]:
bq_store.similarity_search(
    "What should I do when I call the emergency roadside assistance?"
)

[Document(page_content="manual.md 2024-03-23\n21 / 22Wash your vehicle regularly to remove dirt and grime.\nWax your vehicle twice a year to protect the paint.\nCheck the tire pressure regularly and adjust it as needed.\nInspect the brakes regularly for wear and tear.\nKeep the interior of your vehicle clean and free of debris.\nBy following these tips, you can help to keep your Cymbal Starlight 2024 in top condition for many years to\ncome.\nChapter 18: Emergencies\nRoadside Assistance\nIf you experience a roadside emergency, such as a flat tire or a dead battery, you can call roadside\nassistance for help. Roadside assistance is available 24 hours a day, 7 days a week.\nTo call roadside assistance, dial the following number:\n1-800-555-1212\nWhen you call roadside assistance, be prepared to provide the following information:\nYour name and contact information\nYour vehicle's make, model, and year\nYour vehicle's location\nThe nature of the emergency\nFlat Tire", metadata={'doc_id': '

### Get a langchain retriever
The retriever will be used in a Langchain Chain to find the most similar documents for a given query.

In [6]:
langchain_retriever = bq_store.as_retriever()

### Compose a Langchain Chain

We are going to use the [`RetrievalQA` chain](https://python.langchain.com/docs/modules/chains/popular/vector_db_qa)
There are several different chain types available, listed [here](https://docs.langchain.com/docs/components/chains/index_related_chains).

In [7]:
%%time
from langchain_google_vertexai import VertexAI
from langchain.chains import RetrievalQA
from langchain.globals import set_debug

# Set high verbosity
set_debug(True)

llm = VertexAI(model_name="gemini-pro")

search_query = "What should I do when call the emergency roadside assistance?"  # @param {type:"string"}

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=langchain_retriever
)
response = retrieval_qa.invoke(search_query)
print("\n################ Final Answer ################\n")
print(response["result"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What should I do when call the emergency roadside assistance?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What should I do when call the emergency roadside assistance?",
  "context": "manual.md 2024-03-23\n21 / 22Wash your vehicle regularly to remove dirt and grime.\nWax your vehicle twice a year to protect the paint.\nCheck the tire pressure regularly and adjust it as needed.\nInspect the brakes regularly for wear and tear.\nKeep the interior of your vehicle clean and free of debris.\nBy following these tips, you can help to keep your Cymbal Starlight 2024 in top condition for many years to\ncome.\nChapter 18: Emergencies\nRoadside Assistance\nIf you experience a roadside emergency, such as a flat tire or a dead battery, y

## Low latency Vector Search with FeatureStore

We are now ready to perform low latency serving with Feature Store! 

To do that, you can simply use the method `.to_vertex_fs_vector_store()`, to get a `VertexFSVectorStore` object

See the [function definition](TODO) for all the parameters you can use.

Note: Any method we run earlier can be equivalently called on both `BigQueryVectorStore` and `VertexFSVectorStore`. For instance it is possible to add new documents to an instance of `VertexFSVectorStore` as both stores share the same underlying BQ source.

In [8]:
vertex_fs = bq_store.to_vertex_fs_vector_store() # pass optional parameters here

BigQuery table cloud-llm-preview2.vertex_documentation.mytest99 initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=cloud-llm-preview2&ws=!1m5!1m4!4m3!1scloud-llm-preview2!2svertex_documentation!3smytest99
VertexFSVectorStore initialized with Feature Store
online store type: optimized
Optional batch serving available via .to_big_query_vector_store() method.


### Alternatively you can also init the VertexFSVectorStore class directly

In [9]:
vertex_fs = VertexFSVectorStore(
    project_id=PROJECT_ID,
    location=REGION,
    dataset_name=DATASET,
    table_name=TABLE,
    embedding=embedding_model,
    # pass optional parameters here
 )

BigQuery table cloud-llm-preview2.vertex_documentation.mytest99 initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=cloud-llm-preview2&ws=!1m5!1m4!4m3!1scloud-llm-preview2!2svertex_documentation!3smytest99
VertexFSVectorStore initialized with Feature Store
online store type: optimized
Optional batch serving available via .to_big_query_vector_store() method.


#### Kick off a synchronization process

We use the `sync` method to synchronize the data from BigQuery to the Feature Online Store, to achieve low latency serving.

When in a production environment, you can also use `cron_schedule` to setup an automatic scheduled synchronization. 

The synchronization process will take around ~20 minutes.

In [ ]:
vertex_fs.sync()

You can monitor the synchronization process from GCP Console: [Vertex AI Feature Store Tab](https://console.cloud.google.com/vertex-ai/feature-store/online-stores)

#### Serve with Feature Online Store

You are now ready to serve with Feature Store!

In [10]:
langchain_retriever = vertex_fs.as_retriever()

In [21]:
%%time
results = langchain_retriever.invoke("Leaks under the vehicle")
results

CPU times: user 12.2 ms, sys: 2.54 ms, total: 14.7 ms
Wall time: 462 ms


[Document(page_content="manual.md 2024-03-23\n2 / 22VSC can be turned off by pressing the VSC OFF button on the dashboard. However, it is\nrecommended to leave VSC on for optimal safety.\nAnti-Lock Braking System (ABS)\nABS prevents the wheels from locking during braking, allowing you to maintain control of the vehicle.\nABS can be felt as a pulsation in the brake pedal during braking. Do not release the brake pedal;\ncontinue applying steady pressure until the vehicle comes to a stop.\nTire Safety\nMaintain proper tire pressure at all times (see the Tire Pressure Information label on the driver's door\njamb).\nCheck tire tread depth regularly and replace tires when they reach the minimum tread depth of 2/32\ninches.\nAvoid sudden starts, stops, and turns that can cause excessive tire wear.\nVehicle Inspection\nInspect your vehicle regularly for any signs of damage or malfunction, including:\nLeaks under the vehicle\nUnusual noises or vibrations\nDim or flickering lights\nWorn or damag

In [16]:
%%time
response = retrieval_qa.invoke(search_query)
print("\n################ Final Answer ################\n")
print(response["result"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What should I do when call the emergency roadside assistance?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What should I do when call the emergency roadside assistance?",
  "context": "manual.md 2024-03-23\n21 / 22Wash your vehicle regularly to remove dirt and grime.\nWax your vehicle twice a year to protect the paint.\nCheck the tire pressure regularly and adjust it as needed.\nInspect the brakes regularly for wear and tear.\nKeep the interior of your vehicle clean and free of debris.\nBy following these tips, you can help to keep your Cymbal Starlight 2024 in top condition for many years to\ncome.\nChapter 18: Emergencies\nRoadside Assistance\nIf you experience a roadside emergency, such as a flat tire or a dead battery, y

### Filtering by metadata

It is possible to post-filter results by metadata by passing the filter parameter to any search method

VertexFSVectorStore also support metadata filter while performing search, for this to work:
- the `filter_columns` parameter must be passed to `VertexFSVectorStore` when the online feature store feature view is created (first time the class is initialised with a given online store name and feature view name).

- the `string_filters` parameter must be passed to any search method. Note only string fields are supported at the moment. See [here](https://github.com/googleapis/python-aiplatform/blob/8a4a41afe47aaff2f69a73e5011b34bcba5cd2e9/google/cloud/aiplatform_v1beta1/types/feature_online_store_service.py#L345) 


In [17]:
# perform post search filtering
vertex_fs.similarity_search(search_query, filter={'chunk': 56})

[Document(page_content="manual.md 2024-03-23\n21 / 22Wash your vehicle regularly to remove dirt and grime.\nWax your vehicle twice a year to protect the paint.\nCheck the tire pressure regularly and adjust it as needed.\nInspect the brakes regularly for wear and tear.\nKeep the interior of your vehicle clean and free of debris.\nBy following these tips, you can help to keep your Cymbal Starlight 2024 in top condition for many years to\ncome.\nChapter 18: Emergencies\nRoadside Assistance\nIf you experience a roadside emergency, such as a flat tire or a dead battery, you can call roadside\nassistance for help. Roadside assistance is available 24 hours a day, 7 days a week.\nTo call roadside assistance, dial the following number:\n1-800-555-1212\nWhen you call roadside assistance, be prepared to provide the following information:\nYour name and contact information\nYour vehicle's make, model, and year\nYour vehicle's location\nThe nature of the emergency\nFlat Tire", metadata={'source': '

# Appendix

### Local Bruteforce

You can also prototype by using a (local) bruteforce executor. During initialization, data is downloaded from BQ to your memory.

You can use it for prototyping when the number of documents is low. 

### Max Marginal Relevance

In [25]:
mmr_retriever = vertex_fs.as_retriever(search_type="mmr")
mmr_retriever.invoke(search_query)

[Document(page_content="manual.md 2024-03-23\n21 / 22Wash your vehicle regularly to remove dirt and grime.\nWax your vehicle twice a year to protect the paint.\nCheck the tire pressure regularly and adjust it as needed.\nInspect the brakes regularly for wear and tear.\nKeep the interior of your vehicle clean and free of debris.\nBy following these tips, you can help to keep your Cymbal Starlight 2024 in top condition for many years to\ncome.\nChapter 18: Emergencies\nRoadside Assistance\nIf you experience a roadside emergency, such as a flat tire or a dead battery, you can call roadside\nassistance for help. Roadside assistance is available 24 hours a day, 7 days a week.\nTo call roadside assistance, dial the following number:\n1-800-555-1212\nWhen you call roadside assistance, be prepared to provide the following information:\nYour name and contact information\nYour vehicle's make, model, and year\nYour vehicle's location\nThe nature of the emergency\nFlat Tire", metadata={'source': '

### Get documents by ID

You can also use the function `get_documents` to retrieve a set of documents given a document ID:


In [20]:
vertex_fs.get_documents(ids=["6470d610abbd40e2af3c32ede29e09d5"])

[Document(page_content='Car Care Tips\nIn addition to following the maintenance schedule, there are a number of things you can do to keep your\nCymbal Starlight 2024 in good condition. These include:', metadata={'source': 'github-repo/generative-ai/sample-apps/', 'document_name': 'cymbal-starlight-2024.pdf', 'chunk': '55'})]

### Remove documents by ID

You can also use the function `delete` to remove a set of documents given a document ID:

In [ ]:
vertex_fs.delete(ids=["my_id1", "my_id2"])